In [1]:
from sklearn.linear_model import LogisticRegression
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
from azureml.core.workspace import Workspace
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath

In [2]:
datastore_path = "https://raw.githubusercontent.com/thomascjw30/ML-Engineer-Capstone-Project/main/WA_Fn-UseC_-Telco-Customer-Churn.csv"
ds = TabularDatasetFactory.from_delimited_files(path=datastore_path)
df =ds.to_pandas_dataframe()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null bool
Dependents          7043 non-null bool
tenure              7043 non-null int64
PhoneService        7043 non-null bool
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null bool
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7032 non-null float64
Churn               7043 non-null bool
dtypes: bool(5), float64(2), int64(2), obj

In [4]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,32.371149,64.761692,2283.300441
std,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.850000,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


In [5]:
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,True,False,1,False,No phone service,DSL,No,...,No,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False
1,5575-GNVDE,Male,0,False,False,34,True,No,DSL,Yes,...,Yes,No,No,No,One year,False,Mailed check,56.95,1889.50,False
2,3668-QPYBK,Male,0,False,False,2,True,No,DSL,Yes,...,No,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True
3,7795-CFOCW,Male,0,False,False,45,False,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,9237-HQITU,Female,0,False,False,2,True,No,Fiber optic,No,...,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True


In [6]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

### Removing Nulls

In [7]:
df.isna().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [8]:
df = ds.to_pandas_dataframe().dropna()

In [9]:
df.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [10]:
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,True,False,1,False,No phone service,DSL,No,...,No,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False
1,5575-GNVDE,Male,0,False,False,34,True,No,DSL,Yes,...,Yes,No,No,No,One year,False,Mailed check,56.95,1889.50,False
2,3668-QPYBK,Male,0,False,False,2,True,No,DSL,Yes,...,No,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True
3,7795-CFOCW,Male,0,False,False,45,False,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,9237-HQITU,Female,0,False,False,2,True,No,Fiber optic,No,...,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True


In [11]:
df.MultipleLines.unique()

array(['No phone service', 'No', 'Yes'], dtype=object)

In [40]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [45]:
def clean_data(data):
    
    # Dict for cleaning data
    i_service = {"DSL":1, "Fiber optic":2, "No":3}
    multi_lines = {"No phone service":1, "No":2, "Yes":3}
    contract = {'Month-to-month':1, 'One year':2, 'Two year':3}
    payment_method = {'Electronic check':1, 'Mailed check':2, 'Bank transfer (automatic)':3,'Credit card (automatic)':4}
    
    
    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    x_df["gender"] = x_df.gender.apply(lambda s: 1 if s == "Male" else 0)
    x_df["partner"] = x_df.Partner.apply(lambda s: 1 if s == True else 0)
    x_df["dependents"] = x_df.Dependents.apply(lambda s: 1 if s == True else 0)
    x_df["phoneservice"] = x_df.PhoneService.apply(lambda s: 1 if s == True else 0)
    x_df["onlinesecurity"] = x_df.OnlineSecurity.apply(lambda s: 1 if s == 'Yes' else 0)
    x_df["onlinebackup"] = x_df.OnlineBackup.apply(lambda s: 1 if s == 'Yes' else 0)
    x_df["deviceprotection"] = x_df.DeviceProtection.apply(lambda s: 1 if s == 'Yes' else 0)
    x_df["techsupport"] = x_df.TechSupport.apply(lambda s: 1 if s == 'Yes' else 0)
    x_df["streamingtv"] = x_df.StreamingTV.apply(lambda s: 1 if s == 'Yes' else 0)
    x_df["streamingmovies"] = x_df.StreamingMovies.apply(lambda s: 1 if s == 'Yes' else 0)
    x_df["paperbilling"] = x_df.PaperlessBilling.apply(lambda s: 1 if s == True else 0)
    
    x_df['internetservice'] = x_df.InternetService.map(i_service)
    x_df['multiplelines'] = x_df.MultipleLines.map(multi_lines)
    x_df['contract'] = x_df.Contract.map(contract)
    x_df['paymentmethod'] = x_df.PaymentMethod.map(payment_method)
    
    y_df = x_df[['Churn']]
    y_df["churn"] = y_df.Churn.apply(lambda s: 1 if s == True else 0)
    
    x_df.drop(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'], inplace=True, axis=1)

    
    return x_df, y_df

In [46]:
x, y = clean_data(ds)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [47]:
x

,partner,dependents,phoneservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,paperbilling,internetservice,multiplelines,contract,paymentmethod
0,1,0,0,0,1,0,0,0,0,1,1,1,1,1
1,0,0,1,1,0,1,0,0,0,0,1,2,2,2
2,0,0,1,1,1,0,0,0,0,1,1,2,1,2
3,0,0,0,1,0,1,1,0,0,0,1,1,2,3
4,0,0,1,0,0,0,0,0,0,1,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,1,1,1,0,1,1,1,1,1,1,3,2,2
7039,1,1,1,0,1,1,0,1,1,1,2,3,2,4
7040,1,1,0,1,0,0,0,0,0,1,1,1,1,1
7041,1,0,1,0,0,0,0,0,0,1,2,3,1,2
